In [4]:
import pickle
import sys
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import artm
print artm.version()

from os import path, mkdir
from datetime import datetime
%matplotlib inline
sys.path.insert(0, '..\\modules\\helpers')
from plot_helper import PlotMaker
from config_helper import ConfigPaths
import print_helper as ph

0.8.1


In [5]:
config = ConfigPaths('config.cfg')
plot_maker = PlotMaker()
print config.models_file_name
print config.dataset_path 

Q:\\topic_modeling\\csi_science_collections.git\experiments\UCI_filtered_ngramm_trimmed_without_names\np_01_04_model_to_ndw\models.txt
Q:\\topic_modeling\\csi_science_collections.git\..\data\postnauka\UCI_collections\UCI_filtered_ngramm_trimmed_without_names


In [11]:
def get_docs_lenths(_file_name):
    with codecs.open(_file_name, 'r', 'utf-8') as fin:
        # skip comment line
        fin.readline()  
        fin.readline()  
        fin.readline()  
        doc_len_counters = {}
        for line in fin:
            line_list = line.split(' ')
            doc_idx = float(line_list[0])
            val = float(line_list[2])
            if not doc_len_counters.has_key(doc_idx):
                doc_len_counters[doc_idx] = 0
            doc_len_counters[doc_idx] += val
    return doc_len_counters
def save_pickle_file(dists, filename):
    pickle_filename = path.join(config.experiment_path, filename)
    pickle_file = open(pickle_filename, 'wb')
    pickle.dump(dists, pickle_file)
    pickle_file.close()
def load_pickle_file(filename):
    pickle_filename = path.join(config.experiment_path, filename)
    pickle_file = open(pickle_filename, 'rb')
    p_file = pickle.load(pickle_file)
    pickle_file.close()
    return p_file
def save_model_pickle(_model_name, _model, _save=True):
    phi = _model.get_phi()
    phi = phi[(phi.T != 0).any()]
    theta = _model.get_theta()    
    saved_top_tokens = _model.score_tracker['top_tokens_score'].last_tokens
    if _save:
        save_pickle_file(phi, 'phi_{}.p'.format(_model_name))
        save_pickle_file(theta, 'theta_{}.p'.format(_model_name))
        save_pickle_file(saved_top_tokens, 'saved_top_tokens_{}.p'.format(_model_name))
    return phi, theta, saved_top_tokens
def load_model_pickle(_model_name, _distance_name):
    phi = load_pickle_file('phi_{}.p'.format(_model_name))
    theta = load_pickle_file('theta_{}.p'.format(_model_name))
    distances = None
    if _distance_name != None:
        distances = load_pickle_file('{}.p'.format(_distance_name))
    return phi, theta, distances
def convert_to_vw(_nw, out_file):
    with codecs.open(out_file, 'w', 'utf-8') as fout:
        for idx, col in enumerate(_nw.columns):
            if idx % 100 == 0:
                print '[{}] processing column no {} of {}'.format(datetime.now(), idx, len(_nw.columns))
            fout.write(u'doc_{} |@default_class '.format(col))
            values = _nw[col]
            for idx in values.index.values:
                val = values[idx]
                if val != 0:
                    fout.write(u'{}:{} '.format(idx, val))
            fout.write(u'\n')
def create_sample_dataset_multiply_doc_length(phi, theta, doc_lenths):
    ndw = phi.dot(theta)
    index = 0
    for col in ndw.columns:
        if index % 100 == 0:
            print '[{}] processing {} column'.format(datetime.now(), index)
        index += 1
        ndw[col] = ndw[col] * doc_lenths[col]
    return ndw
def create_sample_dataset(phi, theta, doc_lenths):
    ndw = phi.dot(theta)
    return ndw

In [12]:
doc_len_counters = get_docs_lenths(path.join(config.dataset_path, 'docword.pn.txt'))
phi1, theta1, _ = load_model_pickle('model1', None)
phi2, theta2, _ = load_model_pickle('model2', None)
phi3, theta3, _ = load_model_pickle('model3', None)

In [14]:
print phi1.shape, phi2.shape, phi3.shape

(17492, 100) (8051, 100) (2216, 100)


In [9]:
ndw1 = create_sample_dataset_multiply_doc_length(phi1, theta1, doc_len_counters)
ndw2 = create_sample_dataset_multiply_doc_length(phi2, theta2, doc_len_counters)
ndw3 = create_sample_dataset_multiply_doc_length(phi3, theta3, doc_len_counters)

[2017-01-18 18:09:14.182000] processing 0 column
[2017-01-18 18:09:32.636000] processing 100 column
[2017-01-18 18:09:49.117000] processing 200 column
[2017-01-18 18:10:06.488000] processing 300 column
[2017-01-18 18:10:22.782000] processing 400 column
[2017-01-18 18:10:38.955000] processing 500 column
[2017-01-18 18:10:55.160000] processing 600 column
[2017-01-18 18:11:09.700000] processing 700 column
[2017-01-18 18:11:22.596000] processing 800 column
[2017-01-18 18:11:35.037000] processing 900 column
[2017-01-18 18:11:47.019000] processing 1000 column
[2017-01-18 18:11:58.557000] processing 1100 column
[2017-01-18 18:12:09.642000] processing 1200 column
[2017-01-18 18:12:20.197000] processing 1300 column
[2017-01-18 18:12:30.355000] processing 1400 column
[2017-01-18 18:12:40.034000] processing 1500 column
[2017-01-18 18:12:49.207000] processing 1600 column
[2017-01-18 18:12:58.016000] processing 1700 column
[2017-01-18 18:13:06.355000] processing 1800 column
[2017-01-18 18:13:14.179

In [ ]:
ndw10 = create_sample_dataset(phi1, theta1, doc_len_counters)
ndw20 = create_sample_dataset(phi2, theta2, doc_len_counters)
ndw30 = create_sample_dataset(phi3, theta3, doc_len_counters)

In [10]:
output_vw_path = path.join(config.home_dir, '..\\data\postnauka\\UCI_collections', 'sample_model_ndw')
if not path.exists(output_vw_path):
    mkdir(output_vw_path)
convert_to_vw(ndw1, path.join(output_vw_path, 'pn_model1.vw'))
convert_to_vw(ndw2, path.join(output_vw_path, 'pn_model2.vw'))
convert_to_vw(ndw3, path.join(output_vw_path, 'pn_model3.vw'))

[2017-01-18 18:14:21.809000] processing column no 0 of 3446
[2017-01-18 18:15:07.224000] processing column no 100 of 3446
[2017-01-18 18:15:51.818000] processing column no 200 of 3446
[2017-01-18 18:16:37.440000] processing column no 300 of 3446
[2017-01-18 18:17:23.427000] processing column no 400 of 3446
[2017-01-18 18:18:07.943000] processing column no 500 of 3446
[2017-01-18 18:18:52.553000] processing column no 600 of 3446
[2017-01-18 18:19:36.892000] processing column no 700 of 3446
[2017-01-18 18:20:20.838000] processing column no 800 of 3446
[2017-01-18 18:21:05.563000] processing column no 900 of 3446
[2017-01-18 18:21:50.052000] processing column no 1000 of 3446
[2017-01-18 18:22:34.108000] processing column no 1100 of 3446
[2017-01-18 18:23:19.985000] processing column no 1200 of 3446
[2017-01-18 18:24:04.088000] processing column no 1300 of 3446
[2017-01-18 18:24:48.234000] processing column no 1400 of 3446
[2017-01-18 18:25:32.232000] processing column no 1500 of 3446
[201

In [ ]:
convert_to_vw(ndw2, path.join(output_vw_path, 'pn_model2.vw'))
output_vw_path = path.join(config.home_dir, '..\\data\postnauka\\UCI_collections', 'sample_model_ndw_pure')
if not path.exists(output_vw_path):
    mkdir(output_vw_path)
convert_to_vw(ndw10, path.join(output_vw_path, 'pn_model1.vw'))
convert_to_vw(ndw20, path.join(output_vw_path, 'pn_model2.vw'))
convert_to_vw(ndw30, path.join(output_vw_path, 'pn_model3.vw'))